In [14]:
import os
import sys
sys.path.insert(0, '/kaggle/input/linear_mlp/pytorch/default/1')

In [15]:
!pip install --no-index --find-links=/kaggle/input/library/pytorch/default/3 pqdm

Looking in links: /kaggle/input/library/pytorch/default/3


In [16]:
!pip install --no-index --find-links=/kaggle/input/library/pytorch/default/3 astropy

Looking in links: /kaggle/input/library/pytorch/default/3


In [17]:
from utility import predict_fuc, load_best_model,cal_flux, predict_spectra,postprocessing
from linear_MLP import MLP
from linear_process import preprocess_pro
import pandas as pd
import numpy as np
import torch

In [18]:
predictions_spectra_test = preprocess_pro('test')

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

Hot pixel mask shape: (32, 32)


COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

Hot pixel mask shape: (32, 356)


COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
wave_alpha_test = np.array([
    cal_flux(predictions_spectra_test[i])
    for i in range(len(predictions_spectra_test))
])
whitelight_s_test = np.array([
    predict_spectra(predictions_spectra_test[i])
    for i in range(len(predictions_spectra_test))
])  # 预测每个星球的白光缩放比例S
b = 0.007255
a = 0.000375
whitelight_s_test_normalized = (whitelight_s_test - a) / (b - a)
whitelight_s_test_reshaped = whitelight_s_test_normalized[:, np.newaxis]
data_test_reshaped = np.concatenate(
    [np.stack(wave_alpha_test),
     np.stack(whitelight_s_test_reshaped)],
    axis=1  # 按第 2 维度（axis=1）进行合并，以便后续一起处理
)
data_test_reshaped = torch.tensor(data_test_reshaped).float()

In [20]:
batch_size = 1
model = MLP().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model, optimizer, best_val_loss, target_min, target_max = load_best_model(
    model, optimizer,
    '/kaggle/input/linear_mlp/pytorch/default/1/best_model_LR_v1.0.pth')

Best model loaded.


/kaggle/input/linear_mlp/pytorch/default/1/utility.py:122: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


In [21]:
all_predictions = predict_fuc(model, data_test_reshaped, batch_size=1)
all_predictions = all_predictions * (target_max - target_min) + target_min
all_predictions = all_predictions.numpy()

In [22]:
wavelengths = pd.read_csv(
    '/kaggle/input/ariel-data-challenge-2024/wavelengths.csv')
test_adc_info = pd.read_csv(
    '/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
    index_col='planet_id')
train_labels = pd.read_csv(
    '/kaggle/input/ariel-data-challenge-2024/train_labels.csv',
    index_col='planet_id')

In [23]:
sub_df = postprocessing(all_predictions,
                        wavelengths,
                        test_adc_info.index,
                        sigma_pred=np.tile(
                            np.where(test_adc_info[['star']] <= 1, 0.000067,
                                     0.001), (1, 283)))

In [24]:
display(sub_df)

,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,wl_10,...,sigma_274,sigma_275,sigma_276,sigma_277,sigma_278,sigma_279,sigma_280,sigma_281,sigma_282,sigma_283
planet_id,,,,,,,,,,,,,,,,,,,,,
499191466,0.002625,0.002648,0.002654,0.002641,0.002627,0.002629,0.002617,0.00264,0.002634,0.002616,...,0.000067,0.000067,0.000067,0.000067,0.000067,0.000067,0.000067,0.000067,0.000067,0.000067


In [25]:
sub_df.to_csv('/kaggle/working/submission.csv', index=True)